# A Machine Learning Approach Towards Customer Lead Classification

## 1. Import Modules

In [242]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pylab as pl
import warnings

In [243]:
from numpy import set_printoptions
from sklearn.preprocessing import MinMaxScaler
from numpy import set_printoptions
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from matplotlib import rcParams
from matplotlib.cm import rainbow
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics

In [3]:
%pylab

warnings.filterwarnings('ignore')

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


## 2. The Data

### 2.1 Read CSV file

In [4]:
data = pd.read_csv('Desktop/vierde-jaar/1-industrial-project/0-data/leads-data-full.csv', sep=';', low_memory=False)

### 2.2 Display top of data frame

In [5]:
data.head()

,Id,Topic,ec_leadsource,ec_leadsource_label,ec_assessmentifnecessary,ec_assessmentifnecessary_label,CSC Source,CSC Source Label,CSC Notes,CSC Notes Label,...,ownerid_entitytype,createdon,modifiedon,Import Date,Follow Up Date 1,Follow Up Date 2,Follow Up Date 3,Follow Up Date 4,Merged,Merged_label
0,22C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,systemuser,2020/05/29 14:24,2021/06/28 11:26,2020/05/29 14:24,NaN,NaN,NaN,NaN,False,No
1,24C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,systemuser,2020/05/29 14:24,2021/06/28 11:26,2020/05/29 14:24,NaN,NaN,NaN,NaN,False,No
2,26C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,systemuser,2020/05/29 14:24,2021/06/28 11:28,2020/05/29 14:24,NaN,NaN,NaN,NaN,False,No
3,28C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,systemuser,2020/05/29 14:24,2021/06/28 11:26,2020/05/29 14:24,NaN,NaN,NaN,NaN,False,No
4,2AC2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,100000000.0,Completed,NaN,NaN,NaN,NaN,...,systemuser,2020/05/29 14:24,2020/10/05 21:16,2020/05/29 14:24,NaN,NaN,NaN,NaN,False,No


## 3. Data Exploration, Analysis & Visualisation

### 3.1 Data dimensions and types

In [6]:
data.shape

(87456, 97)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87456 entries, 0 to 87455
Data columns (total 97 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Id                                          87456 non-null  object 
 1   Topic                                       87207 non-null  object 
 2   ec_leadsource                               84033 non-null  float64
 3   ec_leadsource_label                         84033 non-null  object 
 4   ec_assessmentifnecessary                    8525 non-null   float64
 5   ec_assessmentifnecessary_label              8525 non-null   object 
 6   CSC Source                                  23644 non-null  float64
 7   CSC Source Label                            23644 non-null  object 
 8   CSC Notes                                   27278 non-null  float64
 9   CSC Notes Label                             27278 non-null  object 
 10  ec_aboveth

### 3.2 Send target feature to back

In [8]:
target = ['LeadRating', 'Lead Rating label']
data = data[[c for c in data if c not in target] 
        + [c for c in target if c in data]]
data.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/lead-rating-data.csv', index=False)

### 3.3 Replace NA with NaN and convert necessary data types to 'float'

In [9]:
data = data.replace({'Not Available': np.nan})

In [10]:
data.head()

,Id,Topic,ec_leadsource,ec_leadsource_label,ec_assessmentifnecessary,ec_assessmentifnecessary_label,CSC Source,CSC Source Label,CSC Notes,CSC Notes Label,...,modifiedon,Import Date,Follow Up Date 1,Follow Up Date 2,Follow Up Date 3,Follow Up Date 4,Merged,Merged_label,LeadRating,Lead Rating label
0,22C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,2021/06/28 11:26,2020/05/29 14:24,NaN,NaN,NaN,NaN,False,No,2.0,Warm
1,24C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,2021/06/28 11:26,2020/05/29 14:24,NaN,NaN,NaN,NaN,False,No,2.0,Warm
2,26C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,2021/06/28 11:28,2020/05/29 14:24,NaN,NaN,NaN,NaN,False,No,2.0,Warm
3,28C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,2021/06/28 11:26,2020/05/29 14:24,NaN,NaN,NaN,NaN,False,No,2.0,Warm
4,2AC2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,100000000.0,Completed,NaN,NaN,NaN,NaN,...,2020/10/05 21:16,2020/05/29 14:24,NaN,NaN,NaN,NaN,False,No,1.0,Hot


In [11]:
data['ec_sendgeneralmarketingmaterial'] = data['ec_sendgeneralmarketingmaterial'].astype(float)
data['Academic check'] = data['Academic check'].astype(float)
data['Credit check'] = data['Credit check'].astype(float)
data['Merged'] = data['Merged'].astype(float)

data['statecode'] = data['statecode'].astype(float)
data['statuscode'] = data['statuscode'].astype(float)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87456 entries, 0 to 87455
Data columns (total 97 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Id                                          87456 non-null  object 
 1   Topic                                       87207 non-null  object 
 2   ec_leadsource                               84033 non-null  float64
 3   ec_leadsource_label                         84033 non-null  object 
 4   ec_assessmentifnecessary                    8525 non-null   float64
 5   ec_assessmentifnecessary_label              8525 non-null   object 
 6   CSC Source                                  23644 non-null  float64
 7   CSC Source Label                            23644 non-null  object 
 8   CSC Notes                                   27278 non-null  float64
 9   CSC Notes Label                             27278 non-null  object 
 10  ec_aboveth

### 3.4 Missing values identification

In [13]:
def missing_values_table(data):
    mis_val = data.isnull().sum()
    mis_val_percent = 100 * data.isnull().sum() / len(data)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
    return mis_val_table_ren_columns

In [14]:
mv_data = missing_values_table(data)

In [15]:
mv_data.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/mv-data.csv', index = True)

In [16]:
# Description of data to be used for missing value imputation:
description = data.describe()
description.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/data-description.csv', index=True)

### 3.5 Drop columns with a high missing value percentage

In [17]:
perc = 80.0
min_count =  int(((100-perc)/100)*data.shape[0] + 1)
data = data.dropna( axis=1, thresh=min_count)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87456 entries, 0 to 87455
Data columns (total 54 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Id                                       87456 non-null  object 
 1   Topic                                    87207 non-null  object 
 2   ec_leadsource                            84033 non-null  float64
 3   ec_leadsource_label                      84033 non-null  object 
 4   CSC Source                               23644 non-null  float64
 5   CSC Source Label                         23644 non-null  object 
 6   CSC Notes                                27278 non-null  float64
 7   CSC Notes Label                          27278 non-null  object 
 8   ec_belowthelineadvertising               24460 non-null  float64
 9   ec_belowthelineadvertising_label         24460 non-null  object 
 10  ec_campaigns                             22645

In [19]:
data.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/mv-dropped-data.csv', index=False)

### 3.6 Plot target class distribution

In [20]:
plt.style.use('fivethirtyeight')
plt.hist(data['Lead Rating label'].dropna(), bins = 3, edgecolor = 'k');
plt.xlabel('Lead Rating'); plt.ylabel('Number of Entries');

In [21]:
lead_ratings = data.groupby('Lead Rating label').size()
print(lead_ratings)

Lead Rating label
Cold    10544
Hot     13064
Warm    16094
dtype: int64


### 3.7 Exploring correlations

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87456 entries, 0 to 87455
Data columns (total 54 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Id                                       87456 non-null  object 
 1   Topic                                    87207 non-null  object 
 2   ec_leadsource                            84033 non-null  float64
 3   ec_leadsource_label                      84033 non-null  object 
 4   CSC Source                               23644 non-null  float64
 5   CSC Source Label                         23644 non-null  object 
 6   CSC Notes                                27278 non-null  float64
 7   CSC Notes Label                          27278 non-null  object 
 8   ec_belowthelineadvertising               24460 non-null  float64
 9   ec_belowthelineadvertising_label         24460 non-null  object 
 10  ec_campaigns                             22645

In [23]:
data.plot(kind='density', subplots=True, layout=(4,5), sharex=False)
plt.show()

In [24]:
data['Campaign Name'].value_counts()

Wavemaker                       7200
Wavemaker Digital Campaign 2    1319
Facebook and Google Search      1195
Wavemaker Media Campaign        1017
facebook and google ads          814
                                ... 
No Increase Gr R fees 2021         1
enrolments                         1
Metis FB Aug-Nov 2020              1
faacebook and google ads           1
wavemaker campaign                 1
Name: Campaign Name, Length: 654, dtype: int64

In [25]:
## Density plot for a single 'object' categorical feature Campaign Name, relative to the Lead Rating:

types = data.dropna(subset=['LeadRating'])
types = types['Campaign Name'].value_counts()
types = list(types[types.values > 300].index)

# Plot of distribution of Lead Ratings for Campaign Name categories
figsize(12, 10)

# Plot each Campaign
for c_type in types:
    # Select the Campaign type
    subset = data[data['Campaign Name'] == c_type]
    
    # Density plot of Lead Rating labels
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
# label the plot
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by Campaign name', size = 28);
plt.legend(loc="upper left");

In [26]:
data['What is important you as a parent label'].value_counts()

Quality Of Education      17478
Security And Safety        3067
Other                      2660
Extra Mural Activities      192
Name: What is important you as a parent label, dtype: int64

In [27]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['What is important you as a parent label'].value_counts()
types = list(types[types.values > 100].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['What is important you as a parent label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by Parent Importance Features', size = 28);
plt.legend(loc="upper left");

In [28]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['ec_relationshiptolearner_label'].value_counts()
types = list(types[types.values > 100].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['ec_relationshiptolearner_label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by Relationship to Learner', size = 28);
plt.legend(loc="upper left");

In [29]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['ec_belowthelineadvertising_label'].value_counts()
types = list(types[types.values > 1000].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['ec_belowthelineadvertising_label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by Below-the-line Advertising', size = 28);
plt.legend(loc="upper left");

In [30]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['ec_whatareyoulookingfor_label'].value_counts()
types = list(types[types.values > 50].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['ec_whatareyoulookingfor_label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by What Looking For', size = 28);
plt.legend(loc="upper left");

In [31]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['ec_campaigns_label'].value_counts()
types = list(types[types.values > 20].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['ec_campaigns_label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by Marketing Campaigns', size = 28);
plt.legend(loc="upper left");

In [381]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['CSC Notes Label'].value_counts()
types = list(types[types.values > 20].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['CSC Notes Label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by CSC Notes', size = 28);
plt.legend(loc="upper left");

In [382]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['CSC Source Label'].value_counts()
types = list(types[types.values > 20].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['CSC Source Label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by CSC Source', size = 28);
plt.legend(loc="upper left");

In [383]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['Boarding School Needed label'].value_counts()
types = list(types[types.values > 20].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['Boarding School Needed label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by Boarding School Needed', size = 28);
plt.legend(loc="upper left");

In [384]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['School Fee Discount Applicable label'].value_counts()
types = list(types[types.values > 20].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['School Fee Discount Applicable label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by School Fee Discount', size = 28);
plt.legend(loc="upper left");

In [385]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['Enrolment Discount Applicable label'].value_counts()
types = list(types[types.values > 20].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['Enrolment Discount Applicable label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by Enrolment Discount', size = 28);
plt.legend(loc="upper left");

In [386]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['Aftercare Needed label'].value_counts()
types = list(types[types.values > 20].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['Aftercare Needed label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by Aftercare Needed', size = 28);
plt.legend(loc="upper left");

In [388]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['ec_leadsource_label'].value_counts()
types = list(types[types.values > 20].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['ec_leadsource_label'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by Lead Source', size = 28);
plt.legend(loc="upper left");

In [390]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['statuscode'].value_counts()
types = list(types[types.values > 2000].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['statuscode'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by Status Code', size = 28);
plt.legend(loc="upper left");

In [391]:
## For a single 'object' categorical feature, relative to the Lead Rating

types = data.dropna(subset=['LeadRating'])
types = types['statecode'].value_counts()
types = list(types[types.values > 2000].index)

figsize(12, 10)

for c_type in types:

    subset = data[data['statecode'] == c_type]
    
    sns.kdeplot(subset['LeadRating'].dropna(), label = c_type, shade = False, alpha = 0.8);
    
plt.xlabel('Lead Rating', size = 20); plt.ylabel('Density', size = 20); 
plt.title('Density Plot of Lead Ratings by State Code', size = 28);
plt.legend(loc="upper left");

In [32]:
## Correlations between all numerical features and the target:
correlations_data = data.corr()['LeadRating'].sort_values()
correlations_data

Credit check                        -0.454316
Academic check                      -0.452883
CSC Notes                           -0.062013
ec_belowthelineadvertising          -0.060150
Enrolment Discount Applicable       -0.050441
ec_whatareyoulookingfor             -0.038027
Merged                              -0.034302
CSC Source                          -0.029020
ec_campaigns                        -0.021711
ec_gradeofenrolment                  0.022009
Boarding School Needed               0.028911
ec_relationshiptolearner             0.052484
School Fee Discount Applicable       0.100427
Aftercare Needed                     0.123333
statuscode                           0.177359
ec_sendgeneralmarketingmaterial      0.180107
ec_leadsource                        0.231722
What is important you as a parent    0.243678
statecode                            0.398259
LeadRating                           1.000000
Name: LeadRating, dtype: float64

In [33]:
## Pairs Plot of Correlations:

# Extract the columns to  plot
plot_data = data[['LeadRating', 'Credit check', 
                      'School Fee Discount Applicable', 
                      'statecode']]

# Replace the inf with nan
plot_data = plot_data.replace({np.inf: np.nan, -np.inf: np.nan})

# Rename columns 
# plot_data = plot_data.rename(columns = {'Site EUI (kBtu/ft²)': 'Site EUI', 
#                                        'Weather Normalized Source EUI (kBtu/ft²)': 'Weather Norm EUI',
#                                        'log_Total GHG Emissions (Metric Tons CO2e)': 'log GHG Emissions'})

# Drop na values
plot_data = plot_data.dropna()

# Function to calculate correlation coefficient between two columns
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size = 20)

# Create the pairgrid object
grid = sns.PairGrid(data = plot_data, height = 3)

# Upper is a scatter plot
grid.map_upper(plt.scatter, color = 'red', alpha = 0.6)

# Diagonal is a histogram
grid.map_diag(plt.hist, color = 'red', edgecolor = 'black')

# Bottom is correlation and density plot
grid.map_lower(corr_func);
grid.map_lower(sns.kdeplot, cmap = plt.cm.Reds)

# Title for entire plot
plt.suptitle('Pairs Plot of Lead Rating', size = 36, y = 1.02);

### 3.8 Skewness

In [34]:
skewness = data.skew()
print(skewness)

ec_leadsource                        -0.134356
CSC Source                            0.347461
CSC Notes                             3.560175
ec_belowthelineadvertising            0.206287
ec_campaigns                          8.064552
ec_whatareyoulookingfor               6.870314
ec_sendgeneralmarketingmaterial       0.123085
ec_relationshiptolearner              1.814713
ec_gradeofenrolment                  -0.187990
Aftercare Needed                     -2.709741
Boarding School Needed               -4.660195
What is important you as a parent     1.353538
Enrolment Discount Applicable         3.739759
School Fee Discount Applicable       -5.593703
Academic check                        2.689040
Credit check                          2.699190
statecode                            -0.925492
statuscode                           -0.002150
Merged                               12.399310
LeadRating                            0.108500
dtype: float64


### 3.9 Histograms

In [35]:
data.hist()
plt.show()

### 3.10 Density plots

In [36]:
data.plot(kind='density', subplots=True, layout=(5,4), sharex=False)
plt.show()

### 3.11 Box and whisker

In [37]:
data.plot(kind='box', subplots=True, layout=(6,6), sharex=False, sharey=False)
plt.show()

### 3.12 Correlation matrix plot

In [38]:
correlations = data.corr(method='pearson')
correlations.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/correlations.csv', index=True)

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
names = correlations.columns
ticks = np.arange(0,20,1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(names, rotation = 70)
ax.set_yticklabels(names)
fig.colorbar(cax)
plt.show()

### 3.13 Scatter plot matrix

In [40]:
scatter_matrix(data)
plt.show()

## 4. Data Preparation

### 4.1 Split dataset according to Lead Rating entry

In [41]:
training_data = data[data.LeadRating.notnull()]
training_data.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/training-data.csv', index=False)

In [42]:
training_data.shape

(39702, 54)

In [43]:
training_data.head()

,Id,Topic,ec_leadsource,ec_leadsource_label,CSC Source,CSC Source Label,CSC Notes,CSC Notes Label,ec_belowthelineadvertising,ec_belowthelineadvertising_label,...,statuscode_label,ownerid,ownerid_entitytype,createdon,modifiedon,Import Date,Merged,Merged_label,LeadRating,Lead Rating label
0,22C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,No longer interested,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2021/06/28 11:26,2020/05/29 14:24,0.0,No,2.0,Warm
1,24C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,No longer interested,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2021/06/28 11:26,2020/05/29 14:24,0.0,No,2.0,Warm
2,26C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,Affordability,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2021/06/28 11:28,2020/05/29 14:24,0.0,No,2.0,Warm
3,28C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,No longer interested,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2021/06/28 11:26,2020/05/29 14:24,0.0,No,2.0,Warm
4,2AC2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,Enrolled,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2020/10/05 21:16,2020/05/29 14:24,0.0,No,1.0,Hot


In [44]:
prediction_data = data[data.LeadRating.isna()]
prediction_data.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/prediction-data.csv', index=False)

In [45]:
prediction_data.shape

(47754, 54)

In [46]:
prediction_data.head()

,Id,Topic,ec_leadsource,ec_leadsource_label,CSC Source,CSC Source Label,CSC Notes,CSC Notes Label,ec_belowthelineadvertising,ec_belowthelineadvertising_label,...,statuscode_label,ownerid,ownerid_entitytype,createdon,modifiedon,Import Date,Merged,Merged_label,LeadRating,Lead Rating label
13,3CC2123A-A7A1-EA11-A811-000D3A49DE7B,New Lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,No longer interested,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2021/06/28 11:22,2020/05/29 14:24,0.0,No,NaN,NaN
26,56C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2020/11/20 10:28,2020/05/29 14:24,0.0,No,NaN,NaN
59,264BBC37-A8A1-EA11-A811-000D3A49DE7B,New lead,100000002.0,Online/Social Media,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,0318F045-18C4-E911-A86A-000D3A45A317,systemuser,2020/05/29 14:31,2020/11/20 10:28,2020/05/29 14:31,0.0,No,NaN,NaN
60,2A4BBC37-A8A1-EA11-A811-000D3A49DE7B,New lead,100000002.0,Online/Social Media,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,0318F045-18C4-E911-A86A-000D3A45A317,systemuser,2020/05/29 14:31,2020/11/20 10:28,2020/05/29 14:31,0.0,No,NaN,NaN
61,2C4BBC37-A8A1-EA11-A811-000D3A49DE7B,New lead,100000002.0,Online/Social Media,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,0318F045-18C4-E911-A86A-000D3A45A317,systemuser,2020/05/29 14:31,2020/11/20 10:28,2020/05/29 14:31,0.0,No,NaN,NaN


### 4.2 Rescale data

### 4.3 Standardise data

### 4.4 Normalise data 

### 4.5 Binarise data

### 4.6 Handling missing values:

In [47]:
## Use iterative imputer for MV imputation:
td = training_data.select_dtypes(include=['float'])

vals = td.values
ix = [i for i in range(vals.shape[1]) if i != 19]
X, y = vals[:, ix], vals[:, 19]

# summarize total missing
print('Missing: %d' % sum(isnan(X).flatten()))

# define imputer
imputer = IterativeImputer()

# fit on the dataset
imputer.fit(X)

# transform the dataset
Xtrans = imputer.transform(X)

# summarize new total missing
print('Missing: %d' % sum(isnan(Xtrans).flatten()))

Missing: 237707
Missing: 0


In [48]:
td.columns

Index(['ec_leadsource', 'CSC Source', 'CSC Notes',
       'ec_belowthelineadvertising', 'ec_campaigns', 'ec_whatareyoulookingfor',
       'ec_sendgeneralmarketingmaterial', 'ec_relationshiptolearner',
       'ec_gradeofenrolment', 'Aftercare Needed', 'Boarding School Needed',
       'What is important you as a parent', 'Enrolment Discount Applicable',
       'School Fee Discount Applicable', 'Academic check', 'Credit check',
       'statecode', 'statuscode', 'Merged', 'LeadRating'],
      dtype='object')

In [49]:
training_data = pd.DataFrame(Xtrans, columns = ['ec_leadsource', 'CSC Source', 'CSC Notes',
       'ec_belowthelineadvertising', 'ec_campaigns', 'ec_whatareyoulookingfor',
       'ec_sendgeneralmarketingmaterial', 'ec_relationshiptolearner',
       'ec_gradeofenrolment', 'Aftercare Needed', 'Boarding School Needed',
       'What is important you as a parent', 'Enrolment Discount Applicable',
       'School Fee Discount Applicable', 'Academic check', 'Credit check',
       'statecode', 'statuscode', 'Merged'])

In [50]:
training_data

,ec_leadsource,CSC Source,CSC Notes,ec_belowthelineadvertising,ec_campaigns,ec_whatareyoulookingfor,ec_sendgeneralmarketingmaterial,ec_relationshiptolearner,ec_gradeofenrolment,Aftercare Needed,Boarding School Needed,What is important you as a parent,Enrolment Discount Applicable,School Fee Discount Applicable,Academic check,Credit check,statecode,statuscode,Merged
0,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,100000016.0,0.0
1,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,100000016.0,0.0
2,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,100000005.0,0.0
3,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,100000016.0,0.0
4,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.0,1.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39697,100000001.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000001.0,0.0
39698,100000001.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.0,1.0,3.0,0.0
39699,100000004.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000017.0,0.0
39700,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000001.0,0.0


In [51]:
td

,ec_leadsource,CSC Source,CSC Notes,ec_belowthelineadvertising,ec_campaigns,ec_whatareyoulookingfor,ec_sendgeneralmarketingmaterial,ec_relationshiptolearner,ec_gradeofenrolment,Aftercare Needed,Boarding School Needed,What is important you as a parent,Enrolment Discount Applicable,School Fee Discount Applicable,Academic check,Credit check,statecode,statuscode,Merged,LeadRating
0,100000003.0,NaN,NaN,NaN,100000001.0,NaN,1.0,NaN,NaN,100000001.0,100000001.0,100000001.0,100000000.0,100000002.0,0.0,0.0,2.0,100000016.0,0.0,2.0
1,100000003.0,NaN,NaN,NaN,100000001.0,NaN,1.0,NaN,NaN,100000001.0,100000001.0,100000001.0,100000000.0,100000002.0,0.0,0.0,2.0,100000016.0,0.0,2.0
2,100000003.0,NaN,NaN,NaN,100000001.0,NaN,1.0,NaN,NaN,100000001.0,100000001.0,100000001.0,100000000.0,100000002.0,0.0,0.0,2.0,100000005.0,0.0,2.0
3,100000003.0,NaN,NaN,NaN,100000001.0,NaN,1.0,NaN,NaN,100000001.0,100000001.0,100000001.0,100000000.0,100000002.0,0.0,0.0,2.0,100000016.0,0.0,2.0
4,100000003.0,NaN,NaN,NaN,100000001.0,NaN,1.0,NaN,100000007.0,100000001.0,100000001.0,100000001.0,100000000.0,100000002.0,1.0,1.0,1.0,3.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87450,100000001.0,NaN,NaN,100000017.0,NaN,NaN,1.0,NaN,NaN,100000001.0,100000001.0,100000001.0,100000000.0,100000002.0,0.0,0.0,0.0,100000001.0,0.0,2.0
87452,100000001.0,NaN,NaN,100000009.0,NaN,100000000.0,0.0,100000000.0,100000014.0,100000001.0,100000001.0,100000001.0,100000001.0,100000000.0,1.0,1.0,1.0,3.0,0.0,1.0
87453,100000004.0,100000014.0,100000004.0,100000021.0,NaN,100000000.0,0.0,NaN,100000007.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,100000017.0,0.0,1.0
87454,100000002.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,100000014.0,100000001.0,100000001.0,100000001.0,100000000.0,100000002.0,0.0,0.0,0.0,100000001.0,0.0,2.0


In [52]:
LeadRating_list = td['LeadRating'].values
LeadRating_list

array([2., 2., 2., ..., 1., 2., 1.])

In [53]:
training_data['Lead Rating'] = LeadRating_list

In [54]:
training_data

,ec_leadsource,CSC Source,CSC Notes,ec_belowthelineadvertising,ec_campaigns,ec_whatareyoulookingfor,ec_sendgeneralmarketingmaterial,ec_relationshiptolearner,ec_gradeofenrolment,Aftercare Needed,Boarding School Needed,What is important you as a parent,Enrolment Discount Applicable,School Fee Discount Applicable,Academic check,Credit check,statecode,statuscode,Merged,Lead Rating
0,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,100000016.0,0.0,2.0
1,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,100000016.0,0.0,2.0
2,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,100000005.0,0.0,2.0
3,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,100000016.0,0.0,2.0
4,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.0,1.0,3.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39697,100000001.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000001.0,0.0,2.0
39698,100000001.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.0,1.0,3.0,0.0,1.0
39699,100000004.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000017.0,0.0,1.0
39700,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000001.0,0.0,2.0


In [55]:
training_data.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/adjusted-training-data.csv', index=False)

In [56]:
training_data.shape

(39702, 20)

## 5. Feature Selection

### 5.1 Categorical feature scoring

In [77]:
# load the dataset
def load_dataset(dataframe):
    # retrieve numpy array
    dataset = dataframe.values
    # split into input (X) and output (y) variables
    X = dataset[:, :-1]
    y = dataset[:,-1]
    # format all fields as string
    X = X.astype(str)
    return X, y

In [78]:
# prepare input data
def prepare_inputs(X_train, X_test):
    oe = OrdinalEncoder(handle_unknown='ignore')
    oe.fit(X_train)
    X_train_enc = oe.transform(X_train)
    X_test_enc = oe.transform(X_test)
    return X_train_enc, X_test_enc

# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    return y_train_enc, y_test_enc

# feature selection X^2
def select_features_X(X_train, y_train, X_test):
    fs = SelectKBest(score_func=chi2, k='all')
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

# feature selection Mutual Info
def select_features_MI(X_train, y_train, X_test):
    fs = SelectKBest(score_func=mutual_info_classif, k='all')
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

In [79]:
## X^2 METHOD

# load the dataset
X, y = load_dataset(training_data)

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)

# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

# feature selection
X_train_fs, X_test_fs, fs1 = select_features_X(X_train_enc, y_train_enc, X_test_enc)

# what are scores for the features
for i in range(len(fs1.scores_)):
    print('Feature %d: %f' % (i, fs1.scores_[i]))

# plot the scores
pyplot.bar([i for i in range(len(fs1.scores_))], fs1.scores_)
pyplot.show()
pyplot.title("Chi-Squared Test")
pyplot.xlabel('Index of Descriptive Feature')
pyplot.ylabel('Feature Importance Score')

Feature 0: 26704.634575
Feature 1: 597687.945291
Feature 2: 1558984.812793
Feature 3: 430.497367
Feature 4: 2101784.513207
Feature 5: 212537.303181
Feature 6: 2001557.346160
Feature 7: 30220.414641
Feature 8: 372464.835098
Feature 9: 4389179.992778
Feature 10: 33331.515493
Feature 11: 55277.709917
Feature 12: 3105.530491
Feature 13: 11352.520824


Text(0, 0.5, 'Feature Importance Score')

In [80]:
## MUTUAL INFORMATION METHOD

# load the dataset
X, y = load_dataset(training_data)

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# prepare input data
X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)

# prepare output data
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

# feature selection
X_train_fs, X_test_fs, fs2 = select_features_MI(X_train_enc, y_train_enc, X_test_enc)

# what are scores for the features
for i in range(len(fs2.scores_)):
    print('Feature %d: %f' % (i, fs2.scores_[i]))
    
# plot the scores
pyplot.bar([i for i in range(len(fs2.scores_))], fs2.scores_)
pyplot.show()
pyplot.title("Mutual Information Test")
pyplot.xlabel('Index of Descriptive Feature')
pyplot.ylabel('Feature Importance Score')

Feature 0: 0.061363
Feature 1: 0.239026
Feature 2: 0.214032
Feature 3: 0.229683
Feature 4: 0.190519
Feature 5: 0.151009
Feature 6: 0.215227
Feature 7: 0.155655
Feature 8: 0.154304
Feature 9: 0.197824
Feature 10: 0.052223
Feature 11: 0.172273
Feature 12: 0.246552
Feature 13: 0.281031


Text(0, 0.5, 'Feature Importance Score')

In [82]:
training_data.columns

Index(['ec_leadsource', 'CSC Source', 'CSC Notes',
       'ec_belowthelineadvertising', 'ec_campaigns', 'ec_whatareyoulookingfor',
       'ec_relationshiptolearner', 'Aftercare Needed',
       'Boarding School Needed', 'What is important you as a parent',
       'Enrolment Discount Applicable', 'School Fee Discount Applicable',
       'statecode', 'statuscode', 'Lead Rating'],
      dtype='object')

### 5.2 Feature selection

In [83]:
training_data = training_data.drop(['Merged', 'ec_sendgeneralmarketingmaterial',
                                    'Academic check', 'Credit check', 'ec_gradeofenrolment'], axis = 1)
training_data.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/final-training-data.csv', index=False)

KeyError: "['Merged' 'ec_sendgeneralmarketingmaterial' 'Academic check'\n 'Credit check' 'ec_gradeofenrolment'] not found in axis"

## 6.Establishing a Naive Baseline

### 6.1 Mean Absolute Error (MAE)

In [84]:
# function to calculate mean absolute error
def mae(y_true, y_pred):
    return np.mean(abs(y_true - y_pred))

# load the dataset
X, y = load_dataset(training_data)

# establish naive baseline
baseline_guess = np.median(y)

print('The baseline guess is a lead rating score of %0.2f' % baseline_guess)
print('Baseline Performance on the test set: MAE = %0.4f' % mae(y_test, baseline_guess))

The baseline guess is a lead rating score of 2.00
Baseline Performance on the test set: MAE = 0.5916


## 7. Evaluating Different Machine Learning Models

In [85]:
# Takes in a model, trains the model, and evaluates the model on the test set
def fit_and_evaluate(model):
    
    # Train the model
    model.fit(X, y)
    
    # Make predictions and evalute
    model_pred = model.predict(X_test)
    model_mae = mae(y_test, model_pred)
    
    # Return the performance metric
    return model_mae

### 7.1 Feature scaling: Normalisation

In [87]:
# Create the scaler object with a range of 0-1
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit on the training data
scaler.fit(X)

# Transform both the training and testing data
X = scaler.transform(X)
X_test = scaler.transform(X_test)

### 7.2 Fitting and evaluating different ML models

In [89]:
## LINEAR REGRESSION

lr = LinearRegression()
lr_mae = fit_and_evaluate(lr)

print('Linear Regression Performance on the test set: MAE = %0.4f' % lr_mae)

Linear Regression Performance on the test set: MAE = 0.5459


In [90]:
## SUPPORT VECTOR MACHINE

svm = SVR(C = 10, gamma = 0.1)
svm_mae = fit_and_evaluate(svm)

print('Support Vector Machine Regression Performance on the test set: MAE = %0.4f' % svm_mae)

Support Vector Machine Regression Performance on the test set: MAE = 0.4318


In [91]:
## RANDOM FOREST

random_forest = RandomForestRegressor(random_state=60)
random_forest_mae = fit_and_evaluate(random_forest)

print('Random Forest Regression Performance on the test set: MAE = %0.4f' % random_forest_mae)

Random Forest Regression Performance on the test set: MAE = 0.2374


In [92]:
## GRADIENT BOOSTED REGRESSION

gradient_boosted = GradientBoostingRegressor(random_state=60)
gradient_boosted_mae = fit_and_evaluate(gradient_boosted)

print('Gradient Boosted Regression Performance on the test set: MAE = %0.4f' % gradient_boosted_mae)

Gradient Boosted Regression Performance on the test set: MAE = 0.4471


In [93]:
## K-NEAREST NEIGHBOUR

knn = KNeighborsRegressor(n_neighbors=10)
knn_mae = fit_and_evaluate(knn)

print('K-Nearest Neighbors Regression Performance on the test set: MAE = %0.4f' % knn_mae)

K-Nearest Neighbors Regression Performance on the test set: MAE = 0.3542


### 7.7 Plot comparison of MAE's

In [98]:
plt.style.use('fivethirtyeight')
figsize(8, 6)

# Dataframe to hold the results
model_comparison = pd.DataFrame({'model': ['LR', 'SVM',
                                           'RF', 'GBR',
                                            'kNN'],
                                 'MAE': [lr_mae, svm_mae, random_forest_mae, 
                                         gradient_boosted_mae, knn_mae]})

# Horizontal bar chart of test mae
model_comparison.sort_values('MAE', ascending = False).plot(x = 'model', y = 'MAE', kind = 'barh',
                                                           color = 'blue', edgecolor = 'black')

# Plot formatting
plt.ylabel(''); plt.yticks(size = 14); plt.xlabel('Mean Absolute Error'); plt.xticks(size = 10)
plt.title('Model Comparison of Test MAEs', size = 20);

## 8. Model Implementation

### 8.1 Random forest classification

In [350]:
training_data.columns

Index(['ec_leadsource', 'CSC Source', 'CSC Notes',
       'ec_belowthelineadvertising', 'ec_campaigns', 'ec_whatareyoulookingfor',
       'ec_relationshiptolearner', 'Aftercare Needed',
       'Boarding School Needed', 'What is important you as a parent',
       'Enrolment Discount Applicable', 'School Fee Discount Applicable',
       'statecode', 'statuscode', 'Lead Rating'],
      dtype='object')

In [120]:
#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators=1000)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)

y_pred = numpy.round_(clf.predict(X_test), decimals = 0, out = None)

# Calculate the mean absolute errors
errors1 = abs(y_pred - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors1), 2))

Mean Absolute Error: 0.41


In [114]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred)*100, 2), '%.')

Accuracy: 63.39 %.


In [115]:
y_test

array([3., 2., 1., ..., 2., 2., 1.])

In [107]:
## OR

In [359]:
X, y = load_dataset(training_data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [360]:
# Construct RF model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train);

In [361]:
# Use the forest's predict method on the test data
predictions = numpy.round_(rf.predict(X_test), decimals = 0, out = None)

# Calculate the mean absolute errors
errors = abs(predictions - y_test)

In [362]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display average predictive accuracy
accuracy = 100 - np.mean(mape)
print('Average Predictive Accuracy:', round(accuracy, 2), '%')

Average Predictive Accuracy: 71.93 %


### 8.2 Confusion matrix

In [363]:
cm = confusion_matrix(y_test, predictions)
print(cm)

[[2344 1106  498]
 [ 634 3027 1164]
 [  47  933 2158]]


In [364]:
ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);

# labels, title and ticks
ax.set_xlabel('Predicted labels'); ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Hot', 'Warm', 'Cold']); ax.yaxis.set_ticklabels(['Hot', 'Warm', 'Cold']);

### 8.3 Precision, recall and F1 score

In [164]:
print('Prediction Accuracy:', round(accuracy, 2), '%.')
print(classification_report(y_test,predictions))

Prediction Accuracy: 71.89 %.
              precision    recall  f1-score   support

         1.0       0.76      0.59      0.67      3882
         2.0       0.61      0.63      0.62      4878
         3.0       0.57      0.69      0.62      3151

    accuracy                           0.63     11911
   macro avg       0.65      0.64      0.64     11911
weighted avg       0.65      0.63      0.64     11911



## 9. Hyperparameter Tuning Using GA

### 9.1 Random search k-fold cross validation

In [165]:
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [166]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [172]:
# First create the base model to tune
rf1 = RandomForestClassifier()

# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf1, param_distributions = random_grid, n_iter = 20, cv = 2, verbose=2, random_state=42, n_jobs = -1)

#Fit the random search model
rf_random.fit(X_train, y_train)


Fitting 2 folds for each of 20 candidates, totalling 40 fits


RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [173]:
# Find optimal parameters
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': False}

In [204]:
# Accuracy evalutaion function
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Average Error: {:0.4f}.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

# Accuracy of base model
base_model = RandomForestClassifier(n_estimators = 1000, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

# Accuracy of model with optimised random parameters 
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Average Error: 0.4140.
Accuracy = 71.89%.
Average Error: 0.4154.
Accuracy = 71.85%.
Improvement of -0.06%.


### 9.2 Grid search k-fold cross validation

In [178]:
# Create the parameter grid based on the results of random search 
param_grid = {'n_estimators': [1000, 2000],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'max_features': ['sqrt'],
 'max_depth': [10, 50, 100],
 'bootstrap': [True]}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 2, n_jobs = -1, verbose = 2)

In [180]:
# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 2 folds for each of 54 candidates, totalling 108 fits


{'bootstrap': True,
 'max_depth': 50,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 2000}

In [181]:
# Accuracy of model with optimised parameters 
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

Model Performance
Average Error: 0.4149.
Accuracy = 71.87%.
Improvement of -0.03%.


### 9.3 General k-fold cross validation

In [365]:
# Look at parameters used by the  current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [398]:
# Number of trees used in the boosting process
n_estimators = [100, 500, 900, 1100, 1500]

# Maximum depth of each tree
max_depth = [2, 3, 5, 10, 15]

# Minimum number of samples per leaf
min_samples_leaf = [1, 2, 4, 6, 8]

# Minimum number of samples to split a node
min_samples_split = [2, 4, 6, 10]

# Maximum number of features to consider for making splits
max_features = ['auto', 'sqrt', 'log2', None]

# Define the grid of hyperparameters to search
hyperparameter_grid = {'n_estimators': n_estimators,
                       'max_depth': max_depth,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}

# Create the model to use for hyperparameter tuning
model = RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=model,
                               param_distributions=hyperparameter_grid,
                               cv=4, n_iter=25, 
                               scoring = 'neg_mean_absolute_error',
                               n_jobs = -1, verbose = 1, 
                               return_train_score = True,
                               random_state=42)

# Fit on the training data
random_cv.fit(X_train, y_train)

Fitting 4 folds for each of 25 candidates, totalling 100 fits


RandomizedSearchCV(cv=4,
                   estimator=RandomForestClassifier(n_estimators=1000,
                                                    random_state=42),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'max_depth': [2, 3, 5, 10, 15],
                                        'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 4, 6, 10],
                                        'n_estimators': [100, 500, 900, 1100,
                                                         1500]},
                   random_state=42, return_train_score=True,
                   scoring='neg_mean_absolute_error', verbose=1)

In [399]:
random_cv.best_params_

{'n_estimators': 900,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': None,
 'max_depth': 15}

In [400]:
best_random = random_cv.best_estimator_

In [401]:
# Make predictions on the test set using default and final model
default_pred = rf.predict(X_test)
final_pred = best_random.predict(X_test)

In [402]:
# Evaluate base vs. final model
# Accuracy of base model
default_accuracy = evaluate(base_model, X_test, y_test)

# Accuracy of model with optimised parameters 
final_accuracy = evaluate(best_random, X_test, y_test)

print('Improvement of {:0.2f}%.'.format( 100 * (final_accuracy - default_accuracy) / default_accuracy))

Average Error: 0.4080.
Accuracy = 72.32%.
Average Error: 0.4132.
Accuracy = 71.95%.
Improvement of -0.51%.


In [403]:
default_pred

array([3., 2., 2., ..., 1., 1., 1.])

In [404]:
final_pred

array([3., 2., 2., ..., 1., 1., 1.])

In [405]:
default_pred.shape

(11911,)

In [406]:
final_model = best_random

In [407]:
final_model

RandomForestClassifier(max_depth=15, max_features=None, min_samples_leaf=2,
                       n_estimators=900, random_state=42)

### 9.4 Hyperparameter-optimised Confusion Matrix

In [408]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display average predictive accuracy
accuracy = 100 - np.mean(mape)

In [409]:
final_cm = confusion_matrix(y_test, final_pred)
print(final_cm)

[[2348 1102  498]
 [ 643 3016 1166]
 [  47  921 2170]]


In [410]:
ax = plt.subplot()
sns.heatmap(final_cm, annot=True, fmt='g', ax=ax);

# labels, title and ticks
ax.set_xlabel('Predicted labels'); ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Hot', 'Warm', 'Cold']); ax.yaxis.set_ticklabels(['Hot', 'Warm', 'Cold']);

In [411]:
print('Prediction Accuracy:', round(accuracy, 2), '%.')
print(classification_report(y_test, final_pred))

Prediction Accuracy: 71.93 %.
              precision    recall  f1-score   support

         1.0       0.77      0.59      0.67      3948
         2.0       0.60      0.63      0.61      4825
         3.0       0.57      0.69      0.62      3138

    accuracy                           0.63     11911
   macro avg       0.65      0.64      0.64     11911
weighted avg       0.65      0.63      0.63     11911



## 10. Model Evaluation and Interpretation

### 10.1 Feature importance

In [224]:
fs1.scores_

array([2.67046346e+04, 5.97687945e+05, 1.55898481e+06, 4.30497367e+02,
       2.10178451e+06, 2.12537303e+05, 2.00155735e+06, 3.02204146e+04,
       3.72464835e+05, 4.38917999e+06, 3.33315155e+04, 5.52777099e+04,
       3.10553049e+03, 1.13525208e+04])

In [225]:
# model is the trained model
importances = fs1.scores_

# train_features is the dataframe of training features
feature_list = list(['ec_leadsource', 'CSC Source', 'CSC Notes',
       'ec_belowthelineadvertising', 'ec_campaigns', 'ec_whatareyoulookingfor',
       'ec_relationshiptolearner', 'Aftercare Needed',
       'Boarding School Needed', 'What is important you as a parent',
       'Enrolment Discount Applicable', 'School Fee Discount Applicable',
       'statecode', 'statuscode'])

# Extract the feature importances into a dataframe
feature_results = pd.DataFrame({'feature': feature_list, 'importance': importances})

# Show the top 10 most important
feature_results = feature_results.sort_values('importance', ascending = False).reset_index(drop=True)

feature_results.head(19)

,feature,importance
0,What is important you as a parent,4.389180e+06
1,ec_campaigns,2.101785e+06
2,ec_relationshiptolearner,2.001557e+06
3,CSC Notes,1.558985e+06
4,CSC Source,5.976879e+05
5,Boarding School Needed,3.724648e+05
6,ec_whatareyoulookingfor,2.125373e+05
7,School Fee Discount Applicable,5.527771e+04
8,Enrolment Discount Applicable,3.333152e+04
9,Aftercare Needed,3.022041e+04


In [222]:
# plot the scores
pyplot.bar([i for i in range(len(fs1.scores_))], fs1.scores_)
pyplot.show()

### 10.2 Visualising a single tree*

### 10.3 Prediction with greatest error

In [231]:
# Find the residuals
residuals = abs(final_pred - y_test)
    
# Extract the most wrong prediction
wrong = X_test[np.argmax(residuals), :]

print('Prediction: %0.4f' % np.argmax(residuals))
print('Actual Value: %0.4f' % y_test[np.argmax(residuals)])

Prediction: 24.0000
Actual Value: 1.0000


## 11. Evaluating on the PREDICTION_DATA dataset

In [288]:
data

,Id,Topic,ec_leadsource,ec_leadsource_label,CSC Source,CSC Source Label,CSC Notes,CSC Notes Label,ec_belowthelineadvertising,ec_belowthelineadvertising_label,...,statuscode_label,ownerid,ownerid_entitytype,createdon,modifiedon,Import Date,Merged,Merged_label,LeadRating,Lead Rating label
0,22C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,No longer interested,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2021/06/28 11:26,2020/05/29 14:24,0.0,No,2.0,Warm
1,24C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,No longer interested,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2021/06/28 11:26,2020/05/29 14:24,0.0,No,2.0,Warm
2,26C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,Affordability,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2021/06/28 11:28,2020/05/29 14:24,0.0,No,2.0,Warm
3,28C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,No longer interested,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2021/06/28 11:26,2020/05/29 14:24,0.0,No,2.0,Warm
4,2AC2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,Enrolled,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2020/10/05 21:16,2020/05/29 14:24,0.0,No,1.0,Hot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87451,D551B0AA-3BF1-EB11-BACB-000D3AAEAAD2,FW: scholarship,100000004.0,CSC,NaN,NaN,NaN,NaN,NaN,NaN,...,First Contact,FF7E7C8C-A557-EA11-A812-000D3A49D12E,systemuser,2021/07/30 15:40,2021/07/30 15:40,2021/07/30 15:40,0.0,No,NaN,NaN
87452,EFF35698-AD2A-44FB-90B1-9D7BF73ABA95,2022 inskrywing,100000001.0,Below The Line Advertising,NaN,NaN,NaN,NaN,100000009.0,Referred By A Friend,...,Enrolled,1BAA9974-0868-EA11-A811-000D3A49DE7B,systemuser,2021/07/30 15:58,2021/07/30 15:59,2021/07/30 15:58,0.0,No,1.0,Hot
87453,B5944DF6-764E-487A-AE3C-ED3234596F21,Curro Helderwyk lead,100000004.0,CSC,100000014.0,Campaign,100000004.0,Enrolment 2021,100000021.0,Email enquiries,...,Fourth Contact,FF7E7C8C-A557-EA11-A812-000D3A49D12E,systemuser,2021/07/30 16:18,2021/08/24 10:17,2021/07/30 16:18,0.0,No,1.0,Hot
87454,DCE92545-5BF4-4DB2-B918-D4DBB750B58B,New lead,100000002.0,Online/Social Media,NaN,NaN,NaN,NaN,NaN,NaN,...,School Visit,DE9B8B7C-9D1E-EB11-A813-000D3A49DAF7,systemuser,2021/07/30 16:28,2021/07/30 16:29,2021/07/30 16:28,0.0,No,2.0,Warm


In [289]:
## Specify prediction data
prediction_data = data[data.LeadRating.isna()]
prediction_data.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/prediction-data.csv', index=False)

In [290]:
prediction_data.head()

,Id,Topic,ec_leadsource,ec_leadsource_label,CSC Source,CSC Source Label,CSC Notes,CSC Notes Label,ec_belowthelineadvertising,ec_belowthelineadvertising_label,...,statuscode_label,ownerid,ownerid_entitytype,createdon,modifiedon,Import Date,Merged,Merged_label,LeadRating,Lead Rating label
13,3CC2123A-A7A1-EA11-A811-000D3A49DE7B,New Lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,No longer interested,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2021/06/28 11:22,2020/05/29 14:24,0.0,No,NaN,NaN
26,56C2123A-A7A1-EA11-A811-000D3A49DE7B,New lead,100000003.0,Campaigns,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,7411B4B4-549F-EA11-A812-000D3A49DAF7,systemuser,2020/05/29 14:24,2020/11/20 10:28,2020/05/29 14:24,0.0,No,NaN,NaN
59,264BBC37-A8A1-EA11-A811-000D3A49DE7B,New lead,100000002.0,Online/Social Media,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,0318F045-18C4-E911-A86A-000D3A45A317,systemuser,2020/05/29 14:31,2020/11/20 10:28,2020/05/29 14:31,0.0,No,NaN,NaN
60,2A4BBC37-A8A1-EA11-A811-000D3A49DE7B,New lead,100000002.0,Online/Social Media,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,0318F045-18C4-E911-A86A-000D3A45A317,systemuser,2020/05/29 14:31,2020/11/20 10:28,2020/05/29 14:31,0.0,No,NaN,NaN
61,2C4BBC37-A8A1-EA11-A811-000D3A49DE7B,New lead,100000002.0,Online/Social Media,NaN,NaN,NaN,NaN,NaN,NaN,...,Lost,0318F045-18C4-E911-A86A-000D3A45A317,systemuser,2020/05/29 14:31,2020/11/20 10:28,2020/05/29 14:31,0.0,No,NaN,NaN


In [291]:
## Impute missing values
## Use iterative imputer for MV imputation:
pred = prediction_data.select_dtypes(include=['float'])

vals = pred.values
ix = [i for i in range(vals.shape[1]) if i != 19]
X, y = vals[:, ix], vals[:, 19]

# summarize total missing
print('Missing: %d' % sum(isnan(X).flatten()))

# define imputer
imputer = IterativeImputer()

# fit on the dataset
imputer.fit(X)

# transform the dataset
pred_Xtrans = imputer.transform(X)

# summarize new total missing
print('Missing: %d' % sum(isnan(pred_Xtrans).flatten()))

Missing: 440101
Missing: 0


In [292]:
pred.columns

Index(['ec_leadsource', 'CSC Source', 'CSC Notes',
       'ec_belowthelineadvertising', 'ec_campaigns', 'ec_whatareyoulookingfor',
       'ec_sendgeneralmarketingmaterial', 'ec_relationshiptolearner',
       'ec_gradeofenrolment', 'Aftercare Needed', 'Boarding School Needed',
       'What is important you as a parent', 'Enrolment Discount Applicable',
       'School Fee Discount Applicable', 'Academic check', 'Credit check',
       'statecode', 'statuscode', 'Merged', 'LeadRating'],
      dtype='object')

In [293]:
prediction_data = pd.DataFrame(pred_Xtrans, columns = ['ec_leadsource', 'CSC Source', 'CSC Notes',
       'ec_belowthelineadvertising', 'ec_campaigns', 'ec_whatareyoulookingfor',
       'ec_sendgeneralmarketingmaterial', 'ec_relationshiptolearner',
       'ec_gradeofenrolment', 'Aftercare Needed', 'Boarding School Needed',
       'What is important you as a parent', 'Enrolment Discount Applicable',
       'School Fee Discount Applicable', 'Academic check', 'Credit check',
       'statecode', 'statuscode', 'Merged'])

In [294]:
prediction_data

,ec_leadsource,CSC Source,CSC Notes,ec_belowthelineadvertising,ec_campaigns,ec_whatareyoulookingfor,ec_sendgeneralmarketingmaterial,ec_relationshiptolearner,ec_gradeofenrolment,Aftercare Needed,Boarding School Needed,What is important you as a parent,Enrolment Discount Applicable,School Fee Discount Applicable,Academic check,Credit check,statecode,statuscode,Merged
0,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,100000016.0,0.0
1,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,4.0,0.0
2,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,4.0,0.0
3,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,4.0,0.0
4,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47749,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000000.0,0.0
47750,100000001.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000000.0,0.0
47751,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.0,1.0,3.0,0.0
47752,100000001.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000000.0,0.0


In [295]:
pred

,ec_leadsource,CSC Source,CSC Notes,ec_belowthelineadvertising,ec_campaigns,ec_whatareyoulookingfor,ec_sendgeneralmarketingmaterial,ec_relationshiptolearner,ec_gradeofenrolment,Aftercare Needed,Boarding School Needed,What is important you as a parent,Enrolment Discount Applicable,School Fee Discount Applicable,Academic check,Credit check,statecode,statuscode,Merged,LeadRating
13,100000003.0,NaN,NaN,NaN,100000001.0,NaN,1.0,NaN,100000013.0,NaN,NaN,NaN,100000000.0,NaN,0.0,0.0,2.0,100000016.0,0.0,NaN
26,100000003.0,NaN,NaN,NaN,100000001.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,100000000.0,NaN,0.0,0.0,2.0,4.0,0.0,NaN
59,100000002.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,100000005.0,NaN,NaN,NaN,100000000.0,NaN,0.0,0.0,2.0,4.0,0.0,NaN
60,100000002.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,100000011.0,NaN,NaN,NaN,100000000.0,NaN,0.0,0.0,2.0,4.0,0.0,NaN
61,100000002.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,100000016.0,NaN,NaN,NaN,100000000.0,NaN,0.0,0.0,2.0,4.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87429,100000002.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,100000007.0,NaN,NaN,NaN,100000000.0,NaN,0.0,0.0,0.0,100000000.0,0.0,NaN
87430,100000001.0,NaN,NaN,100000020.0,NaN,100000000.0,1.0,NaN,100000014.0,NaN,NaN,NaN,100000000.0,NaN,0.0,0.0,0.0,100000000.0,0.0,NaN
87437,100000002.0,NaN,NaN,NaN,NaN,NaN,0.0,100000000.0,100000013.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,3.0,0.0,NaN
87441,100000001.0,NaN,NaN,100000020.0,NaN,100000004.0,0.0,100000000.0,100000003.0,NaN,NaN,NaN,100000000.0,NaN,0.0,0.0,0.0,100000000.0,0.0,NaN


In [296]:
## Append empty lead rating column to end of dataset
pred_LR_list = pred['LeadRating'].values
pred_LR_list

array([nan, nan, nan, ..., nan, nan, nan])

In [297]:
prediction_data['Lead Rating'] = pred_LR_list

In [298]:
prediction_data

,ec_leadsource,CSC Source,CSC Notes,ec_belowthelineadvertising,ec_campaigns,ec_whatareyoulookingfor,ec_sendgeneralmarketingmaterial,ec_relationshiptolearner,ec_gradeofenrolment,Aftercare Needed,Boarding School Needed,What is important you as a parent,Enrolment Discount Applicable,School Fee Discount Applicable,Academic check,Credit check,statecode,statuscode,Merged,Lead Rating
0,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,100000016.0,0.0,NaN
1,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,4.0,0.0,NaN
2,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,4.0,0.0,NaN
3,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,4.0,0.0,NaN
4,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,2.0,4.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47749,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000000.0,0.0,NaN
47750,100000001.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000000.0,0.0,NaN
47751,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,1.0,1.0,3.0,0.0,NaN
47752,100000001.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,0.0,0.0,100000000.0,0.0,NaN


In [299]:
## Feature selection based on training_data
prediction_data = prediction_data.drop(['Merged', 'ec_sendgeneralmarketingmaterial',
                                    'Academic check', 'Credit check', 'ec_gradeofenrolment'], axis = 1)
prediction_data.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/final-prediction-data.csv', index=False)

In [339]:
## Load dataset and split to create 100% testing set
X, y = load_dataset(prediction_data)

In [340]:
## Make final predictions on the prediction_data testing set
final_predictions = final_model.predict(X)

In [342]:
## Insert final predictions to lead rating column in prediction dataset
final_predictions

array([3., 3., 3., ..., 1., 2., 2.])

In [343]:
prediction_data['Lead Rating'] = final_predictions

In [344]:
prediction_data

,ec_leadsource,CSC Source,CSC Notes,ec_belowthelineadvertising,ec_campaigns,ec_whatareyoulookingfor,ec_relationshiptolearner,Aftercare Needed,Boarding School Needed,What is important you as a parent,Enrolment Discount Applicable,School Fee Discount Applicable,statecode,statuscode,Lead Rating
0,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,2.0,100000016.0,3.0
1,100000003.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,2.0,4.0,3.0
2,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,2.0,4.0,3.0
3,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,2.0,4.0,3.0
4,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,2.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47749,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,100000000.0,2.0
47750,100000001.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,100000000.0,2.0
47751,100000002.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.0,3.0,1.0
47752,100000001.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,0.0,100000000.0,2.0


In [346]:
## Count number of Hot (1), Warm (2) and Cold (3) leads in the data
prediction_data['Lead Rating'].value_counts()

3.0    22326
2.0    19401
1.0     6027
Name: Lead Rating, dtype: int64

In [345]:
## Save final prediction dataset
prediction_data.to_csv('Desktop/vierde-jaar/1-industrial-project/0-python/final-prediction-data-complete.csv', index=False)